In [88]:
!pip install pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.2 kB 325.1 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 463.6 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ------------------------------

In [132]:
!pip install pynput


   ---------------------------------------- 0.0/90.2 kB ? eta -:--:--
   ------------------ --------------------- 41.0/90.2 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 90.2/90.2 kB 1.0 MB/s eta 0:00:00


In [52]:
import cv2
import mediapipe as mp
import utils
from utils import get_distance, get_angle
import pyautogui
import time
import random
import os
screen_width, screen_height = pyautogui.size()
from pynput.mouse import Button, Controller
mouse = Controller()

In [3]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

In [5]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]  # Assuming only one hand is detected
        index_finger_tip = hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
        return index_finger_tip
    return None, None


In [7]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y / 2 * screen_height)
        pyautogui.moveTo(x, y)

In [9]:
def is_left_click(landmark_list, thumb_index_dist):
    return (
            utils.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            utils.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) > 90 and
            thumb_index_dist > 50
    )

In [11]:
def is_right_click(landmark_list, thumb_index_dist):
    return (
            utils.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            utils.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90  and
            thumb_index_dist > 50
    )

In [13]:
def is_double_click(landmark_list, thumb_index_dist):
    return (
            utils.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            utils.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist > 50
    )

In [32]:
def is_screenshot(landmark_list, thumb_index_dist):
    return (
        utils.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and  
        utils.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and  
        utils.get_angle(landmark_list[13], landmark_list[14], landmark_list[16]) < 50 and  
        thumb_index_dist < 50  # Pouce proche de l'index
    )


In [54]:
def detect_gesture(frame, landmark_list, processed):
    if len(landmark_list) >= 21:
        try:
            index_finger_tip = find_finger_tip(processed)
            thumb_index_dist = utils.get_distance([landmark_list[4], landmark_list[5]])

            if utils.get_distance([landmark_list[4], landmark_list[5]]) < 50 and utils.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90:
                move_mouse(index_finger_tip)
            elif is_left_click(landmark_list, thumb_index_dist):
                mouse.press(Button.left)
                mouse.release(Button.left)
                cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif is_right_click(landmark_list, thumb_index_dist):
                mouse.press(Button.right)
                mouse.release(Button.right)
                cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif is_double_click(landmark_list, thumb_index_dist):
                pyautogui.doubleClick()
                cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            elif is_screenshot(landmark_list, thumb_index_dist):
                print("Screenshot gesture detected - attempting to capture...")
                try:
                    im1 = pyautogui.screenshot()
                    label = random.randint(1, 1000)
                    screenshot_path = f'my_screenshot_{label}.png'
                    im1.save(screenshot_path)
                    print(f"Screenshot saved as: {os.path.abspath(screenshot_path)}")
                    cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
                except Exception as e:
                    print(f"Failed to save screenshot: {e}")
                time.sleep(0.5)  # Prevent multiple screenshots in a loop
        except Exception as e:
            print(f"Error in gesture detection: {e}")

In [56]:
def main(): 
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils
    try:
        while True:  
            ret, frame = cap.read()
            if not ret: 
                break  

            frame = cv2.flip(frame, 1)
            frameRGB= cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)
            landmark_list = []

            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame,hand_landmarks, mpHands.HAND_CONNECTIONS)
                for lm in hand_landmarks.landmark:
                    landmark_list.append((lm.x, lm.y))
                print(landmark_list)

            detect_gesture(frame, landmark_list, processed)
            cv2.imshow('Frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):  
                break
    finally: 
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


[(0.6657953262329102, 1.077705979347229), (0.6364281177520752, 1.0527629852294922), (0.6203251481056213, 0.9747172594070435), (0.6438308358192444, 0.8977965712547302), (0.6707993745803833, 0.8327064514160156), (0.5909948348999023, 0.861960768699646), (0.5714422464370728, 0.744533360004425), (0.5667286515235901, 0.6655436754226685), (0.5648918747901917, 0.5986830592155457), (0.640005886554718, 0.8361990451812744), (0.6192893981933594, 0.7196469306945801), (0.6107407212257385, 0.6345353126525879), (0.6050620675086975, 0.5657824277877808), (0.6945841312408447, 0.840499997138977), (0.698377788066864, 0.8222132325172424), (0.693493127822876, 0.897437572479248), (0.690376341342926, 0.9574325084686279), (0.7483299374580383, 0.8680720329284668), (0.746610164642334, 0.8649212121963501), (0.7332664728164673, 0.9158414006233215), (0.728022575378418, 0.9570431113243103)]
[(0.6922379732131958, 1.08624267578125), (0.6432782411575317, 1.0645068883895874), (0.6088487505912781, 1.0062721967697144), (0.